In [ ]:
## nesne tespitinde çoklu tespitlerin önlenmesini sağlar

import numpy as np
import cv2
from google.colab.patches import cv2_imshow

def non_max_suppression(boxes, probs=None, overlapThresh=0.3):
    # Kutu yoksa boş liste döndür
    if len(boxes) == 0:
        return []

    # Kutuların türünü ondalık sayıya dönüştür
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    # Kutuların koordinatlarını ayır
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    # Kutu alanını hesapla
    area = (x2 - x1 + 1) * (y2 - y1 + 1)

    # İndeksler varsayılan olarak y2'ye göre sıralanır, ancak olasılık değerleri verilmişse ona göre sıralanır
    idxs = y2
    if probs is not None:
        idxs = probs
    idxs = np.argsort(idxs)

    # Seçilen kutuları tutacak liste
    pick = []

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # Karşılaştırma yapılacak kutuları seç
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # Genişlik ve yükseklik hesapla
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        # Örtüşme oranını hesapla
        overlap = (w * h) / area[idxs[:last]]

        # Örtüşme eşiği aşan kutuları ve son kutuyu indekslerden çıkar
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

    # Sonuç olarak seçilen kutuları tamsayı türüne dönüştürüp döndür
    return boxes[pick].astype("int")


In [ ]:
from keras.applications.resnet50 import preprocess_input
from keras.applications import ResNet50
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
import numpy as np
import cv2

## nesne tespitinde çoklu tespitlerin önlenmesini sağlar

import numpy as np
import cv2

### non max suppression ###
def non_max_suppression(boxes, probs=None, overlapThresh=0.3):
    # Kutu yoksa boş liste döndür
    if len(boxes) == 0:
        return []

    # Kutuların türünü ondalık sayıya dönüştür
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    # Kutuların koordinatlarını ayır
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    # Kutu alanını hesapla
    area = (x2 - x1 + 1) * (y2 - y1 + 1)

    # İndeksler varsayılan olarak y2'ye göre sıralanır, ancak olasılık değerleri verilmişse ona göre sıralanır
    idxs = y2
    if probs is not None:
        idxs = probs
    idxs = np.argsort(idxs)

    # Seçilen kutuları tutacak liste
    pick = []

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # Karşılaştırma yapılacak kutuları seç
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # Genişlik ve yükseklik hesapla
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        # Örtüşme oranını hesapla
        overlap = (w * h) / area[idxs[:last]]

        # Örtüşme eşiği aşan kutuları ve son kutuyu indekslerden çıkar
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

    # Sonuç olarak seçilen kutuları tamsayı türüne dönüştürüp döndür
    return boxes[pick].astype("int")

### selective search ###
def selective_search(image):
    print("ss")
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)

    ss.switchToSelectiveSearchQuality()

    rects = ss.process()

    return rects[:1000]


# model
model = ResNet50(weights="imagenet")
image = cv2.imread("//content//drive//MyDrive//rcnn1//animals.jpg")
image = cv2.resize(image, dsize = (400,400))
(H, W) = image.shape[:2]


# ss
rects = selective_search(image)

proposals = []
boxes = []
for (x, y, w, h) in rects:


    if w / float(W) < 0.1 or h / float(H) < 0.1: continue

    roi = image[y:y + h, x:x + w]
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    roi = cv2.resize(roi, (224, 224))

    roi = img_to_array(roi)
    roi = preprocess_input(roi)

    proposals.append(roi)
    boxes.append((x, y, w, h))


proposals = np.array(proposals)


# predict
print("predict")
preds = model.predict(proposals)
preds = imagenet_utils.decode_predictions(preds, top=1)

labels = {}
min_conf = 0.8
for (i, p) in enumerate(preds):


    (_, label, prob) = p[0]
    if prob >= min_conf:
        (x, y, w, h) = boxes[i]
        box = (x, y, x + w, y + h)
        L = labels.get(label, [])
        L.append((box, prob))
        labels[label] = L

clone = image.copy()

for label in labels.keys():
    for (box, prob) in labels[label]:
        boxes = np.array([p[0] for p in labels[label]])
        proba = np.array([p[1] for p in labels[label]])
        boxes = non_max_suppression(boxes, proba)

        for (startX, startY, endX, endY) in boxes:
            cv2.rectangle(clone, (startX, startY), (endX, endY),(0, 0, 255), 2)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.putText(clone, label, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        cv2_imshow(clone)
        if cv2.waitKey(1) & 0xFF == ord('q'):break


Output hidden; open in https://colab.research.google.com to view.